In [ ]:
import os, time
# from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding as sym_padding
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import padding

def to_bytes(x, y=2):
    bytes_x = x.to_bytes(y, byteorder='big')
    return bytes_x

def from_bytes(x, y=2):
    x_num = int.from_bytes(x, 'big')
    return x_num

def client_msg():
    '''
    Session is a random 32 byte number
    Sequence starts off as a 
    
    '''
    session = os.urandom(26)
    sequence = 10000
    seq_bytes = to_bytes(sequence, 2)
    time_stmp = int(time.time())
    time_bytes = time_stmp.to_bytes(4, 'big')
    session_key = session + seq_bytes + time_bytes

    print(f"Session Key: {session_key}")

    with open("server_pubkey", "rb") as f:
        pem_pub = f.read()
    public_key = serialization.load_pem_public_key(pem_pub)

    with open("client_prikey", "rb") as e:
        client_private_key = serialization.load_pem_private_key(
            e.read(),
            password=None
        )
    ciphertext = public_key.encrypt(
        session_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    signed_session = client_private_key.sign(
        ciphertext,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )

    return ciphertext, signed_session



x, y = client_msg()
# print(f"Encrypted message: {x}")
# print(f"Signed message: {y}")

def server_msg(x, y):
    with open("client_pubkey", "rb") as f:
        pem_pub = f.read()
    public_key = serialization.load_pem_public_key(pem_pub)
    public_key.verify(
        y,
        x,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )

    with open("server_prikey", "rb") as e:
        pem_pri = e.read()
    ser_private_key = serialization.load_pem_private_key(pem_pri, password=None)
    plaintext = ser_private_key.decrypt(
        x,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    # Confirming that the messages workd
    print(f"Plaintext: {plaintext}")
    session = plaintext[:26]
    sequence = plaintext[26:28]
    time_stamp = plaintext[28:]

    seq_num = from_bytes(sequence, len(sequence))
    time_stamp_num = from_bytes(time_stamp, len(time_stamp))
    print(seq_num)
    print(time_stamp)

    current_time = int(time.time())
    if current_time - time_stamp_num <= 10:
        print("All good on the time")
    else:
        print("Issue with the time")

    seq_num =+ 1
    sequence_rt = to_bytes(seq_num)
    # current_time_int = int(current_time)
    time_stamp_rt = to_bytes(current_time, 4)

    return_session_key = session + sequence_rt + time_stamp_rt

    ciphertext_return = public_key.encrypt(
        return_session_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    return ciphertext_return

return_msg = server_msg(x, y)
print(return_msg)

Session Key: b"Fb\xdc\xdc\x08\xc3\x9e1\xc5\xb3 \x0c\xc3\n7\xdf\xadvG\xf2\xde;\xc16\xe9='\x10h\t\xba."
Plaintext: b"Fb\xdc\xdc\x08\xc3\x9e1\xc5\xb3 \x0c\xc3\n7\xdf\xadvG\xf2\xde;\xc16\xe9='\x10h\t\xba."
10000
b'h\t\xba.'
All good on the time
b'q\xe4L\x88\xf0N\xda\xc8P\xecgP.\x1d\xa6\xf8+9<\xd1`\xccZ\x10/\xb5t\x84\xeeLo9\xe2\xe7\xdc\xd1\xef\x9e)\xb6\x95O\xa0\xd9\xf5k\x8c0g\x1e\x89\x93\x05ZG\xc2\xafz^A\x1e`c\r\x0c&\xe1\x12Ql,\xec\x91\x9f\x9d\xde\x80[\xcf\xca\xb9\xabt\x0b\xa9\xe4^M\xf7c\xbf\xc2\x18\xa6\xcb\x80{\xba\xc7\xafoD#Z\xe0@\xc69\x15\x07\x01\x90\x8d\xd4\x96\x07Z\xfa\xba\x0b\xc3\xb60\xeb\xa5\xf4n[\xbd\x05\xe9#\xec\x88\xac.\xe0\xf4\xdf\xdd\xdfc\xde\xc4\xe6\xa2\xeaF{\x0ex?\xc0H\x1cnA\xba\n\xa7\xd8\xe3<\xa2/\x9f\x02F^\x9b\xc4"\xb5\xf7`3\xe7\xc648\xdaT\xb4\xc9\x17\xf3\x08\xa7\x9c3\xff\xf4\x0b[\xf6\xa8,\x10R\xeeX\xdc\xd6\xa6\x82$xge\xcb\x06\x01J\xbfR\xe6\xae\xa8o\t\x0c&\x19M\x96\x8d\xbc^\xe4\xc6\xc3\xc4j\x1d.F\xed;8_w\xa5.\xf6\xb3OE\xeb!\xca\xcbCQe7\xce'


## Current Work
- Currently I generate a session key from a 26 byte random number, then add 2 byte sequence number and 4 byte time. Makes a 32 byte session key, I then encrypt it and sign the encryption. I can send all of this to the server.
- Server todo's
  - Take message, check signature - DONE
  - Once confirmed parse out session key - DONE
  - If time is within 10 seconds good 
  - Add one to sequence
  - I probably need to keep squence out of the session key but I can add it to the message so it can properly be tracked.
  - Then keep working towards how the message is read and decrypted.
  - Created helper functions to switch back and forth on ints and bytes.